# HOMEWORK 2

* Work on Google Analytics Customer Revenue Prediction (https://www.kaggle.com/c/ga-customer-revenue-prediction/overview)
* https://docs.google.com/document/d/1cz4u465B4Oi86gLZSoHeEVIHH5BzSGnodHDaIUWCh0w/edit

#### FROM THE SPEC ABOVE (Copied here for easy reference)

1. Take a look at the training data. There may be anomalies in the data that you may need to factor in before you start on the other tasks. Make a note of the anomalies that you notice. Clean the data first to handle these issues. Explain what you did to clean the data (in bulleted form). (10 points)

2. Generate a heatmap and two other plots (with a subset of variables) visualizing interesting positive and negative correlations. Explain the reason for your choice for these variables and any interesting results associated with them. (15 points)

3. Cluster the data based on geographic information available with a subset of variables that you find relevant.  Include a visualization plot.  Describe your inferences from the clustering and discuss their significance. (15 points)

4. Define a buying score or probability function for each user, which predicts the likelihood of a user buying a product from the GStore. Rank the ten most likely users as who will buy a product from the store.  Does it seem that you that it produces good results?  Report why or why not. (15 points)

5. Identify at least one external data set which you can integrate into your transaction prediction analysis to make it better. Discuss/analyze the extent to which this data helps with the prediction task. (10 points).

6. Finally, build the best prediction model you can to solve the Kaggle task.  Use any data, ideas, and approach that you like. Submit the results of your best models on Kaggle.  Report the rank, score, number of entries, for your highest rank. Include a snapshot of your best score on the leaderboard as confirmation. (20 points)

7. Do a permutation test to determine whether your model really benefits from each input variable you use.   In particular, one at a time, for each relevant input variable, permute the value of this variable and see how they impact the accuracy of the results.  Run enough permutations per variable to establish a p-value of how good your predictions of log of sum of transactions per user are.  You can use whatever metric you wish to score your model (like mean absolute error).  (15 points)



## Data Fields


* **fullVisitorId**- A unique identifier for each user of the Google Merchandise Store.
* **channelGrouping** - The channel via which the user came to the Store.
* **date** - The date on which the user visited the Store.
* **device** - The specifications for the device used to access the Store.
* **geoNetwork** - This section contains information about the geography of the user.
* **socialEngagementType** - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
* **totals** - This section contains aggregate values across the session.
* **trafficSource** - This section contains information about the Traffic Source from which the session originated.
* **visitId** - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
* **visitNumber** - The session number for this user. If this is the first session, then this is set to 1.
* **visitStartTime** - The timestamp (expressed as POSIX time).
* **hits** - This row and nested fields are populated for any and all types of hits. Provides a record of all page visits.
* **customDimensions** - This section contains any user-level or session-level custom dimensions that are set for a session. This is a repeated field and has an entry for each dimension that is set.
* **totals** - This set of columns mostly includes high-level aggregate data.
    * **totals.transactionRevenue** - The target variable we want to predict


In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('./homework2_data/train_v2.csv', nrows=1000)

In [8]:
df.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552


In [12]:
for v in df['totals']:
    print(v)

{"visits": "1", "hits": "1", "pageviews": "1", "bounces": "1", "newVisits": "1", "sessionQualityDim": "1"}
{"visits": "1", "hits": "2", "pageviews": "2", "timeOnSite": "28", "sessionQualityDim": "2"}
{"visits": "1", "hits": "2", "pageviews": "2", "timeOnSite": "38", "newVisits": "1", "sessionQualityDim": "1"}
{"visits": "1", "hits": "2", "pageviews": "2", "timeOnSite": "1", "newVisits": "1", "sessionQualityDim": "1"}
{"visits": "1", "hits": "2", "pageviews": "2", "timeOnSite": "52", "newVisits": "1", "sessionQualityDim": "1"}
{"visits": "1", "hits": "2", "pageviews": "2", "timeOnSite": "12", "newVisits": "1", "sessionQualityDim": "2"}
{"visits": "1", "hits": "2", "pageviews": "2", "timeOnSite": "9", "newVisits": "1", "sessionQualityDim": "1"}
{"visits": "1", "hits": "2", "pageviews": "2", "timeOnSite": "15", "newVisits": "1", "sessionQualityDim": "1"}
{"visits": "1", "hits": "2", "pageviews": "2", "timeOnSite": "34", "newVisits": "1", "sessionQualityDim": "1"}
{"visits": "1", "hits": "